In [1]:
import pandas as pd #the csv reading library
import matplotlib.pyplot as plt #the ploting library
import glob #to get file names
import numpy as np
from pandas import Series
import seaborn as sns
from matplotlib.pyplot import figure
import cmocean.cm as cmo
import seaborn as sns

from pylab import rcParams
from numpy import diff
from scipy import interpolate
import scipy.stats as stats

from scipy.signal import savgol_filter
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols

import matplotlib.gridspec as gridspec

import cmocean
import cmocean.cm as cmo
import seaborn as sns

from scipy.interpolate import interp1d

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [2]:
df_meta = pd.read_csv('../Dissertation_projects/Data/meta_data.csv', sep=',', index_col=0)

#fix date time 
df_meta['date_time'] = pd.to_datetime(df_meta['Date'])
df_meta['Date'] = df_meta['date_time'].dt.date

df_meta['code'] = df_meta['Station'] + '_' + df_meta['Trip']

In [3]:
df_spectra_pw = pd.read_csv('../Dissertation_projects/Data/CDOM/sprectra_pchip_baseline_corrected_PW.csv',sep=',', index_col=0)
df_spectra_rz = pd.read_csv('../Dissertation_projects/Data/CDOM/sprectra_pchip_baseline_corrected_RZ.csv',sep=',', index_col=0)

df_spectra_pw.columns = ['a' + col for col in df_spectra_pw.columns]
df_spectra_rz.columns = ['a' + col for col in df_spectra_rz.columns]

df_spectra_pw.index = df_spectra_pw.index.astype(str)
df_spectra_rz.index = df_spectra_rz.index.astype(str)
df_meta.index = df_meta.index.astype(str)

df_spectra_pw = df_spectra_pw.merge(df_meta, left_index=True, right_index=True)
df_spectra_rz = df_spectra_rz.merge(df_meta, left_index=True, right_index=True)

In [4]:
df_c = pd.read_csv('../Dissertation_projects/Data/CDOM/Outputs.csv', sep=',', index_col=0)

# Ensure that the indices are of the same type
df_c.index = df_c.index.astype(str)
df_meta.index = df_meta.index.astype(str)

# Concatenate the DataFrames
df_cdom = df_c.merge(df_meta, left_index=True, right_index=True)

In [5]:
#get PW data sets 
df_cdom_pw = df_cdom[df_cdom['Type'] == 'PW']
 

#get RHZ data 
df_cdom_rz = df_cdom[df_cdom['Type'] == 'RHZ']

df_cdom_rz.columns

Index(['ss_275_700_sem', 'ss_275_295_sem', 'ss_350_400_sem', 'R2_275_700_sem',
       'R2_275_295_sem', 'R2_350_400_sem', 'SR_sem', 'ss_275_700_hm',
       'ss_275_295_hm', 'ss_350_400_hm', 'R2_275_700_hm', 'R2_275_295_hm',
       'R2_350_400_hm', 'SR_hm', 'temp_val', 'temp_rmse', 'mean_700_725',
       'acdom_254', 'acdom_320', 'acdom_350', 'acdom_412', 'acdom_440',
       'acdom_700', 'acdom_254_bc', 'acdom_320_bc', 'acdom_350_bc',
       'acdom_412_bc', 'acdom_440_bc', 'acdom_700_bc', 'file_id', 'Date',
       'Trip', 'Station', 'Lat', 'Lon', 'Water_depth', 'Sample_depth',
       'Sample_depth_units', 'Wavguide_Path_lengths', 'BC_time',
       'BC_Chamber_vol_(L)', 'Type', 'Waveguide_used', 'Notes', 'Flagged',
       'Salinity', 'date_time', 'code'],
      dtype='object')

In [6]:
df_cdom_pw['code'] = df_cdom_pw['Station'] + '_' + df_cdom_pw['Trip']
codes_list_pw = df_cdom_pw['code'].unique()

C:\Users\hmlbr\AppData\Local\Temp\ipykernel_17644\3524013482.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cdom_pw['code'] = df_cdom_pw['Station'] + '_' + df_cdom_pw['Trip']


In [7]:
df_cdom_rz['code'] = df_cdom_rz['Station'] + '_' + df_cdom_rz['Trip']
codes_list_rz = df_cdom_rz['code'].unique()

C:\Users\hmlbr\AppData\Local\Temp\ipykernel_17644\3419403614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cdom_rz['code'] = df_cdom_rz['Station'] + '_' + df_cdom_rz['Trip']


In [8]:
# Group by 'code' and calculate the mean while ignoring NaN values
grouped_df = df_meta.groupby('Station')[['Lat', 'Lon']].mean(numeric_only=True)

# Reset the index to make it a table
grouped_df.reset_index(inplace=True)
df_met = grouped_df

In [9]:
df_por = pd.read_csv('../Dissertation_projects/Data/Porosity/Porosity.csv', sep=',', index_col = 0)
df_por['code'] = df_por['Stn'] + '_' + df_por['camp']

df_p = df_por.groupby('code').agg({
    'Stn': 'first',
    'camp': 'first'
}).reset_index()

df_p.index = df_p['code']

In [10]:
# Group by 'code' and calculate the mean while ignoring NaN values
grouped_df = df_meta.groupby('code')[['Lat', 'Lon']].mean(numeric_only=True)

# Reset the index to make it a table
grouped_df.reset_index(inplace=True)
grouped_df.index = grouped_df['code']
df_met = grouped_df.drop('code', axis = 1)

In [11]:
df_por['code'] = df_por['Stn'] + '_' + df_por['camp']
df_cdom_pw.columns

Index(['ss_275_700_sem', 'ss_275_295_sem', 'ss_350_400_sem', 'R2_275_700_sem',
       'R2_275_295_sem', 'R2_350_400_sem', 'SR_sem', 'ss_275_700_hm',
       'ss_275_295_hm', 'ss_350_400_hm', 'R2_275_700_hm', 'R2_275_295_hm',
       'R2_350_400_hm', 'SR_hm', 'temp_val', 'temp_rmse', 'mean_700_725',
       'acdom_254', 'acdom_320', 'acdom_350', 'acdom_412', 'acdom_440',
       'acdom_700', 'acdom_254_bc', 'acdom_320_bc', 'acdom_350_bc',
       'acdom_412_bc', 'acdom_440_bc', 'acdom_700_bc', 'file_id', 'Date',
       'Trip', 'Station', 'Lat', 'Lon', 'Water_depth', 'Sample_depth',
       'Sample_depth_units', 'Wavguide_Path_lengths', 'BC_time',
       'BC_Chamber_vol_(L)', 'Type', 'Waveguide_used', 'Notes', 'Flagged',
       'Salinity', 'date_time', 'code'],
      dtype='object')

In [12]:

# Assuming df_cdom_pw is your DataFrame and codes_list_pw is a list of codes

# Convert columns to numeric
df_cdom_pw['Sample_depth'] = pd.to_numeric(df_cdom_pw['Sample_depth'], errors='coerce')
df_cdom_pw['acdom_412'] = pd.to_numeric(df_cdom_pw['acdom_412'], errors='coerce')

# Convert columns to numeric
df_cdom_rz['Sample_depth'] = pd.to_numeric(df_cdom_rz['Sample_depth'], errors='coerce')
df_cdom_rz['acdom_412'] = pd.to_numeric(df_cdom_rz['acdom_412'], errors='coerce')



C:\Users\hmlbr\AppData\Local\Temp\ipykernel_17644\4003423132.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cdom_pw['Sample_depth'] = pd.to_numeric(df_cdom_pw['Sample_depth'], errors='coerce')
C:\Users\hmlbr\AppData\Local\Temp\ipykernel_17644\4003423132.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cdom_pw['acdom_412'] = pd.to_numeric(df_cdom_pw['acdom_412'], errors='coerce')
C:\Users\hmlbr\AppData\Local\Temp\ipykernel_17644\4003423132.py:8: SettingWithCopyWarning: 
A value is trying to be 

In [20]:
# Filter for depths between 0 and 5 cm
filtered_df = df_cdom_pw[(df_cdom_pw['Sample_depth'] >= 0) & (df_cdom_pw['Sample_depth'] <= 5)]

# Group by 'code' and calculate the mean 'acdom_412'
mean_a412 = filtered_df.groupby('code')['acdom_412'].mean().reset_index()

# Rename columns for clarity
mean_a412.columns = ['code', 'pw_mean_a412_0_to_5cm']

# Set 'code' as index if required
pw_mean_a412_0_to_5cm = mean_a412.set_index('code', drop=True)
pw_mean_a412_0_to_5cm = pw_mean_a412_0_to_5cm[pw_mean_a412_0_to_5cm['pw_mean_a412_0_to_5cm'] >= 0]

# Optional: Display the result
pw_mean_a412_0_to_5cm

,pw_mean_a412_0_to_5cm
code,
St.11_GOM2021,1.561141
St.11_GOM21FA,2.052190
St.11_GOM22SU,2.766777
St.12_GOM21FA,8.188944
St.12_GOM22SU,5.775122
St.13_GOM21FA,1.389697
St.13_GOM22SP,5.900037
St.13_GOM22SU,7.658755
St.14_GOM2021,16.572027


In [22]:
# Filter for depths between 0 and 5 cm
filtered_df = df_cdom_rz[(df_cdom_rz['Sample_depth'] >= 0) & (df_cdom_rz['Sample_depth'] <= 5)]

# Group by 'code' and calculate the mean 'acdom_412'
mean_a412 = filtered_df.groupby('code')['acdom_412'].mean().reset_index()

# Rename columns for clarity
mean_a412.columns = ['code', 'rz_mean_a412_0_to_5cm']

# Set 'code' as index if required
rz_mean_a412_0_to_5cm = mean_a412.set_index('code', drop=True)

# Optional: Display the result
rz_mean_a412_0_to_5cm

,rz_mean_a412_0_to_5cm
code,
St.11_GOM21FA,-0.897701
St.11_GOM22SU,-1.685262
St.12_GOM21FA,-0.700909
St.13_GOM21FA,1.885674
St.13_GOM22SP,1.060806
St.13_GOM22SU,0.836732
St.14_GOM21FA,-0.642645
St.14_GOM22SP,2.453662
St.14_GOM22SU,1.259384


In [26]:
xx = filtered_df[filtered_df['code'] == 'St.MK_GOM22SP']
xx['acdom_412_bc']

Sample_ID
222262   -1.519581
222262   -1.328076
222262   -1.293617
222263    1.036047
222263   -2.234218
222263    1.053518
222263   -2.229566
222263    1.063614
222263   -2.214819
222264   -0.343832
222264   -0.257267
222264   -0.278481
Name: acdom_412_bc, dtype: float64

In [ ]:
# Filter for depths between 0 and 5 cm
filtered_df = df_cdom_pw[(df_cdom_pw['Sample_depth'] >= 0) & (df_cdom_pw['Sample_depth'] <= 25)]

# Group by 'code' and calculate the mean 'acdom_412'
mean_a412 = filtered_df.groupby('code')['acdom_412'].mean().reset_index()

# Rename columns for clarity
mean_a412.columns = ['code', 'pw_mean_a412_0_to_25cm']

# Set 'code' as index if required
pw_mean_a412_0_to_25cm = mean_a412.set_index('code', drop=True)

# Optional: Display the result
pw_mean_a412_0_to_25cm

In [ ]:
# Filter for depths between 0 and 5 cm
filtered_df = df_cdom_rz[(df_cdom_rz['Sample_depth'] >= 0) & (df_cdom_rz['Sample_depth'] <= 25)]

# Group by 'code' and calculate the mean 'acdom_412'
mean_a412 = filtered_df.groupby('code')['acdom_412'].mean().reset_index()

# Rename columns for clarity
mean_a412.columns = ['code', 'rz_mean_a412_0_to_25cm']

# Set 'code' as index if required
rz_mean_a412_0_to_25cm = mean_a412.set_index('code', drop=True)

# Optional: Display the result
rz_mean_a412_0_to_25cm

In [ ]:
# Filter for depths between 0 and 5 cm
filtered_df = df_cdom_pw[(df_cdom_pw['Sample_depth'] >= 5) & (df_cdom_pw['Sample_depth'] <= 25)]

# Group by 'code' and calculate the mean 'acdom_412'
mean_a412 = filtered_df.groupby('code')['acdom_412'].mean().reset_index()

# Rename columns for clarity
mean_a412.columns = ['code', 'pw_mean_a412_5_to_25cm']

# Set 'code' as index if required
pw_mean_a412_5_to_25cm = mean_a412.set_index('code', drop=True)

# Optional: Display the result
pw_mean_a412_5_to_25cm

In [ ]:
# Filter for depths between 0 and 5 cm
filtered_df = df_cdom_rz[(df_cdom_rz['Sample_depth'] >= 5) & (df_cdom_rz['Sample_depth'] <= 25)]

# Group by 'code' and calculate the mean 'acdom_412'
mean_a412 = filtered_df.groupby('code')['acdom_412'].mean().reset_index()

# Rename columns for clarity
mean_a412.columns = ['code', 'rz_mean_a412_5_to_25cm']

# Set 'code' as index if required
rz_mean_a412_5_to_25cm = mean_a412.set_index('code', drop=True)

# Optional: Display the result
rz_mean_a412_5_to_25cm

In [ ]:
stocks_pw_fe = pd.concat([pw_mean_a412_0_to_5cm, rz_mean_a412_0_to_5cm, 
                          pw_mean_a412_0_to_25cm, rz_mean_a412_0_to_25cm, 
                          pw_mean_a412_5_to_25cm, rz_mean_a412_5_to_25cm], axis=1)
stocks_pw_fe.to_csv('../Dissertation_projects/Data/Stats_data_sets/CDOM_stocks.csv')
stocks_pw_fe